In [13]:
import tensorflow as tf
import cv2
import numpy as np
import random
from collections import deque
import Pong2

In [14]:
ACTIONS = 3
GAMMA = 0.99
INITIAL_EPSILON = 1.0
FINAL_EPSILON = 0.05
EXPLORE = 50000
OBSERVE = 5000
REPLAY_MEMORY = 50000
BATCH = 100

In [15]:
def createGraph1():
    
    W1 = tf.Variable(tf.zeros([8,8,4,32]))
    b1 = tf.Variable(tf.zeros([32]))
    
    W2 = tf.Variable(tf.zeros([4,4,32,64]))
    b2 = tf.Variable(tf.zeros([64]))
    
    W3 = tf.Variable(tf.zeros([3,3,64,64]))
    b3 = tf.Variable(tf.zeros([64]))
    
    W_fc4 = tf.Variable(tf.zeros([3136, 784]))
    b_fc4 = tf.Variable(tf.zeros([784]))
    
    W_fc5 = tf.Variable(tf.zeros([784, ACTIONS]))
    b_fc5 = tf.Variable(tf.zeros([ACTIONS]))
    
    s = tf.placeholder("float", [None, 84, 84, 4])

    conv1 = tf.nn.relu(tf.nn.conv2d(s, W1, strides = [1, 4, 4, 1], padding = "VALID") + b1)
    
    conv2 = tf.nn.relu(tf.nn.conv2d(conv1, W2, strides = [1, 2, 2, 1], padding = "VALID") + b2)
    
    conv3 = tf.nn.relu(tf.nn.conv2d(conv2, W3, strides = [1, 1, 1, 1], padding = "VALID") + b3)
    
    conv3_flat = tf.reshape(conv3, [-1, 3136])
    
    fc4 = tf.nn.relu(tf.matmul(conv3_flat, W_fc4) + b_fc4)
    
    fc5 = tf.matmul(fc4, W_fc5) + b_fc5
    
    return s, fc5
    
    

In [16]:
def createGraph2():
    
    W12 = tf.Variable(tf.zeros([8,8,4,32]))
    b12 = tf.Variable(tf.zeros([32]))
    
    W22 = tf.Variable(tf.zeros([4,4,32,64]))
    b22 = tf.Variable(tf.zeros([64]))
    
    W32 = tf.Variable(tf.zeros([3,3,64,64]))
    b32 = tf.Variable(tf.zeros([64]))
    
    W_fc42 = tf.Variable(tf.zeros([3136, 784]))
    b_fc42 = tf.Variable(tf.zeros([784]))
    
    W_fc52 = tf.Variable(tf.zeros([784, ACTIONS]))
    b_fc52 = tf.Variable(tf.zeros([ACTIONS]))
    
    s2 = tf.placeholder("float", [None, 84, 84, 4])

    conv12 = tf.nn.relu(tf.nn.conv2d(s2, W12, strides = [1, 4, 4, 1], padding = "VALID") + b12)
    
    conv22 = tf.nn.relu(tf.nn.conv2d(conv12, W22, strides = [1, 2, 2, 1], padding = "VALID") + b22)
    
    conv32 = tf.nn.relu(tf.nn.conv2d(conv22, W32, strides = [1, 1, 1, 1], padding = "VALID") + b32)
    
    conv3_flat2 = tf.reshape(conv32, [-1, 3136])
    
    fc42 = tf.nn.relu(tf.matmul(conv3_flat2, W_fc42) + b_fc42)
    
    fc52 = tf.matmul(fc42, W_fc52) + b_fc52
    
    return s2, fc52

In [17]:
def trainGraph(inp1, out1, inp2, out2, sess):
    argmax1 = tf.placeholder("float", [None, ACTIONS])
    argmax2 = tf.placeholder("float", [None, ACTIONS])
    
    gt1 = tf.placeholder("float", [None])
    gt2 = tf.placeholder("float", [None])
    
    action1 = tf.reduce_sum(tf.multiply(out1, argmax1), reduction_indices = 1)
    action2 = tf.reduce_sum(tf.multiply(out2, argmax2), reduction_indices = 1)
    
    cost1 = tf.reduce_mean(tf.square(action1 - gt1))
    cost2 = tf.reduce_mean(tf.square(action2 - gt2))
    
    train_step1 = tf.train.AdamOptimizer(1e-6).minimize(cost1)
    train_step2 = tf.train.AdamOptimizer(1e-6).minimize(cost2)
    
    game = Pong2.PongGame()
    
    D1 = deque()
    D2 = deque()
    
    frame1 = game.getFrame()
    frame2 = game.getFrame()
    
    frame1 = cv2.cvtColor(cv2.resize(frame1, (84, 84)), cv2.COLOR_BGR2GRAY)
    ret1, frame1 = cv2.threshold(frame1, 1, 255, cv2.THRESH_BINARY)
    inp_t1 = np.stack((frame1, frame1, frame1, frame1), axis = 2)
    
    frame2 = cv2.cvtColor(cv2.resize(frame2, (84, 84)), cv2.COLOR_BGR2GRAY)
    ret2, frame2 = cv2.threshold(frame2, 1, 255, cv2.THRESH_BINARY)
    inp_t2 = np.stack((frame2, frame2, frame2, frame2), axis = 2)
    
    sess.run(tf.initialize_all_variables())
    
    t = 0
    epsilon = INITIAL_EPSILON
    
    while(1):
        
        out_t1 = out1.eval(feed_dict = {inp1 : [inp_t1]})[0]
        out_t2 = out2.eval(feed_dict = {inp2 : [inp_t2]})[0]
        
        argmax_t1 = np.zeros([ACTIONS])
        argmax_t2 = np.zeros([ACTIONS])
        
        if(random.random() <= epsilon):
            maxIndex1 = random.randrange(ACTIONS)
            maxIndex2 = random.randrange(ACTIONS)
        else:
            maxIndex1 = np.argmax(out_t1)
            maxIndex2 = np.argmax(out_t2)
        
        argmax_t1[maxIndex1] = 1
        argmax_t2[maxIndex2] = 1
        
        if epsilon > FINAL_EPSILON:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE
        
        frame_n, reward_t1, reward_t2 = game.step_(argmax_t1, argmax_t2)
        
        frame_n = cv2.cvtColor(cv2.resize(frame_n, (84, 84)), cv2.COLOR_BGR2GRAY)
        ret, frame_n = cv2.threshold(frame_n, 1, 255, cv2.THRESH_BINARY)
        
        frame_n1 = np.reshape(frame_n, (84, 84, 1))
        frame_n2 = np.reshape(frame_n, (84, 84, 1))
        
        inp_tt1 = np.append(frame_n1, inp_t1[:, :, 0:3], axis = 2)
        inp_tt2 = np.append(frame_n2, inp_t2[:, :, 0:3], axis = 2)
        
        D1.append((inp_t1, argmax_t1, reward_t1, inp_tt1))
        D2.append((inp_t2, argmax_t2, reward_t2, inp_tt2))
        
        if len(D1) > REPLAY_MEMORY:
            D1.popleft()
        if len(D2) > REPLAY_MEMORY:
            D2.popleft()
        
        if t > OBSERVE:
            
            minibatch1 = random.sample(D1, BATCH)
            inp_batch1 = [d[0] for d in minibatch1]
            argmax_batch1 = [d[1] for d in minibatch1]
            reward_batch1 = [d[2] for d in minibatch1]
            inp_t1_batch1 = [d[3] for d in minibatch1]
            
            minibatch2 = random.sample(D2, BATCH)
            inp_batch2 = [d[0] for d in minibatch2]
            argmax_batch2 = [d[1] for d in minibatch2]
            reward_batch2 = [d[2] for d in minibatch2]
            inp_t1_batch2 = [d[3] for d in minibatch2]
            
            gt_batch1 = []
            gt_batch2 = []
            
            out_batch1 = out1.eval(feed_dict = {inp1 : inp_t1_batch1})
            out_batch2 = out2.eval(feed_dict = {inp2 : inp_t1_batch2})
            
            for i in range(0, len(minibatch1)) :
                gt_batch1.append(reward_batch1[i] + GAMMA * np.max(out_batch1[i]))
            for i in range(0, len(minibatch2)) :
                gt_batch2.append(reward_batch2[i] + GAMMA * np.max(out_batch2[i]))
            
            train_step1.run(feed_dict = {
                           gt1 : gt_batch1,
                           argmax1 : argmax_batch1,
                           inp1 : inp_batch1
                           })
            
            train_step2.run(feed_dict = {
                           gt2 : gt_batch2,
                           argmax2 : argmax_batch2,
                           inp2 : inp_batch2
                           })
            
        inp_t1 = inp_tt1
        inp_t2 = inp_tt2

        if t>OBSERVE :
            t=t
        else:
            t+=1
            

In [18]:
sess = tf.InteractiveSession()
inp1, out1 = createGraph1()
inp2, out2 = createGraph2()
trainGraph(inp1, out1, inp2, out2, sess)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


KeyboardInterrupt: 

In [ ]:
'''
a = Pong2.PongGame()
i = 0
temp = a.reset_()
for i in range(1000):
    act1 = [0,0,0]
    act2 = [0,0,0]
    j = np.random.randint(-1,2)
    act1[j] = 1
    j = np.random.randint(-1,2)
    act2[j] = 1
    temp = a.step_(act1, act2)
    #print(a.paddle1.score," ",a.paddle2.score) 
    
'''